In [9]:
import pandas as pd
import numpy as np

In [10]:
def load_datasets():
    file = '../Data-files/1352S_101F.xlsx'
    sheets = ['整数-101F', 'top5F', 'top3F', '3F']
    tables = {name: pd.read_excel(file, sheet_name=name) for name in sheets}
    for i in tables.values():
        i.index += 1
        
    return tables

In [11]:
def to_otus(raw_tables):
    for name, table in raw_tables.items():
        table = table.drop(columns='Status')
        table.index = table['SampleID']
        table = table.drop(columns='SampleID')
        
        # negative values treated as zeros
        raw_tables[name] = table.applymap(lambda x: x if x >=0 else 0)
    return raw_tables

In [12]:
def make_meta(otus_with_status):
    sources_ix = np.random.rand(1352) < 0.9
    sinks_ix = ~sources_ix
    meta = pd.DataFrame(data={'Env': otus_with_status['Status'].tolist(), 
                              'SourceSink': 'Source'}, 
                        index=otus_with_status['SampleID'])
    meta.iloc[sinks_ix, 1] = 'Sink'
    meta['id'] = 0
    meta[meta['SourceSink'] == 'Sink'].loc[:, 2] = range(1, sum(sinks_ix) + 1)
    return meta

In [13]:
def plot_roc(series):
    #...
    return None

In [36]:
# Define functions
def FEAST_post_proc(df):
    # df = pd.read_csv('../tmp/F3_source_contributions_matrix.txt', sep='\t').T
    df = df.T
    df['GroupID'] = df.index.to_series().apply(lambda x: x.split('_')[1] if x != 'Unknown' else x)
    df = df.groupby(by='GroupID').sum().T
    df['Env'] = df.index.to_series().apply(lambda x: x.split('_')[1])
    df['SampleID'] = df.index.tolist()
    return df

def JSD_post_proc(df, meta):
    # probas = softmax(1-distance) for each sample
    df = 1 - df.T
    df['Env'] = meta.loc[meta['SourceSink']=='Source', 'Env'].tolist()
    softmax = lambda x: np.exp(x) / np.exp(x).sum()
    df = df.groupby(by='Env').mean().apply(softmax, axis=0)
    df = df.T
    df.rename(mapper={'Env': '1', 'died': 'died', 'survived': 'survived'}, axis=1, inplace=True)
    df['Env'] = meta.loc[meta['SourceSink']=='Sink', 'Env'].tolist()
    df['SampleID'] = df.index.tolist()
    return df

# Prepare
names = ["F5_top", "F3_top", "F101", "F3"]
meta = pd.read_csv('../tmp/meta.csv')
res_JSD = {name: '../Tmp/'+name+'.jsd.distance.csv' for name in names}
res_FEAST = {name: '../Tmp/'+name+'_source_contributions_matrix.txt' for name in names}
pred_JSD = {name: '../DLMER-Bio/'+name+'.JSD.predictions.csv' for name in names}
pred_FEAST = {name: '../DLMER-Bio/'+name+'.FEAST.predictions.csv' for name in names}
DLMER_in_JSD = {name: '../DLMER-Bio/'+name+'.JSD.DLMER.in.txt' for name in names}
DLMER_in_FEAST = {name: '../DLMER-Bio/'+name+'.FEAST.DLMER.in.txt' for name in names}

# Read res
res_JSD_dfs = {name: pd.read_csv(f, index_col=0) for name, f in res_JSD.items()}
res_FEAST_dfs = {name: pd.read_csv(f, sep='\t') for name, f in res_FEAST.items()}

JSD_post_proc(res_JSD_dfs['F101'], meta)